<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/YOLO_v5_Wheet_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import tqdm
import torch
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Clone YOLOv5

Clone the YOLOv5 repository, and move all the yolo files to your current working directory

In [2]:
!git clone https://github.com/ultralytics/yolov5
!mv yolov5/* ./

Cloning into 'yolov5'...
remote: Enumerating objects: 15978, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 15978 (delta 89), reused 118 (delta 75), pack-reused 15831
Receiving objects: 100% (15978/15978), 14.61 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (10958/10958), done.


In [3]:
!pip install -r requirements.txt
clear_output()

In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle competitions download global-wheat-detection
!unzip \*.zip && rm *.zip
clear_output()

In [5]:
df = pd.read_csv('/content/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]
df

,image_id,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0
2,b6ab77fd7,377.0,504.0,74.0,160.0,414.0,584.0,0
3,b6ab77fd7,834.0,95.0,109.0,107.0,888.5,148.5,0
4,b6ab77fd7,26.0,144.0,124.0,117.0,88.0,202.5,0
...,...,...,...,...,...,...,...,...
147788,5e0747034,64.0,619.0,84.0,95.0,106.0,666.5,0
147789,5e0747034,292.0,549.0,107.0,82.0,345.5,590.0,0
147790,5e0747034,134.0,228.0,141.0,71.0,204.5,263.5,0
147791,5e0747034,430.0,13.0,184.0,79.0,522.0,52.5,0


In [7]:
# index = list(set(df.image_id))
# len(index)

# source = 'train'
# if True:
#     for fold in [0]:
#         val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
#         for name, mini in tqdm.tqdm(df.groupby('image_id')):
#             if name in val_index:
#                 path2save = 'val2017/'
#             else:
#                 path2save = 'train2017/'
#             if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
#                 os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
#             with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
#                 row = mini[['classes','x_center','y_center','w','h']].astype(float).values
#                 row = row/1024
#                 row = row.astype(str)
#                 for j in range(len(row)):
#                     text = ' '.join(row[j])
#                     f.write(text)
#                     f.write("\n")
#             if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
#                 os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
#             shutil.copy("/content/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))


# index = list(set(df.image_id))  #Each picture contains lots of bounding boxes, so #BBs >> #Img_ids
# print(f'There are {len(index)} Images with {len(df.image_id)} Bounding Boxes')
# source = 'train'
# if True:
#     for fold in [0]:
#         val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
#         print(len(index)*fold//5, len(index)*(fold+1)//5, val_index)
#         for name, mini in tqdm.tqdm(df.groupby('image_id')):
#             path2save = f'convertor/fold{fold}/labels/' + 'val2017' if name in val_index else 'train2017'
#             os.makedirs(path2save, exist_ok=True)

#             with open(f'{path2save}/{name}.txt', 'w+') as f:
#                 row = mini[['classes','x_center','y_center','w','h']].astype(float).values
#                 row = row/1024
#                 row = row.astype(str)
#                 for j in range(len(row)):
#                     text = ' '.join(row[j])
#                     f.write(text)
#                     f.write("\n")
            
#             img2save = f'convertor/fold{fold}/images/'
#             os.makedirs(img2save, exist_ok=True)
#             # os.replace(f"/content/{source}/{name}.jpg", f'{img2save}{name}.jpg')
#             shutil.copy(f"/content/{source}/{name}.jpg", f'{img2save}{name}.jpg')










index = list(set(df.image_id))  #Each picture contains lots of bounding boxes, so #BBs >> #Img_ids
print(f'There are {len(index)} Images with {len(df.image_id)} Bounding Boxes')
source = 'train'
for img_name, bboxs_list in tqdm.tqdm(df.groupby('image_id')):
    label_path = f'convertor/labels/' 
    os.makedirs(label_path, exist_ok=True)

    with open(f'{label_path}/{img_name}.txt', 'w+') as f:
        row = bboxs_list[['classes','x_center','y_center','w','h']].astype(float).values
        row = row/1024
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")
    
    img_path = f'convertor/images/'
    os.makedirs(img_path, exist_ok=True)
    # os.replace(f"/content/{img_name}.jpg", f'{img_path}{img_name}.jpg')
    shutil.copy(f"/content/{img_name}.jpg", f'{img_path}{img_name}.jpg')

There are 3373 Images with 147793 Bounding Boxes
0 674 ['078d5b8b9', '8d9aaa54d', '3fa57b900', '352e238a9', '7bca11467', '2d159a876', '9988ee066', 'cc2bbb768', 'eff99d6e2', 'f7b557230', 'c8e5e50ec', '15d56204e', '266826a59', '9eadd4800', 'fabaeac81', 'f1016e0ea', '4cfb86eff', '499bf6abd', 'ac891ac35', '34b5b6acb', '37b0d5ea3', '060c572e4', 'cf870354f', '86a279fe3', '467dee148', 'b373b765d', '478e21230', '4972b9493', 'f78c551de', 'ed59f37d2', '5a0e2133c', '76be097e9', '1dbc99ed8', '767e515ec', '666e0a853', '8930c88da', '48bdfec34', '4b7c94e8f', '6d1ff0f34', 'd0e1c00bf', 'c233d242d', '93de65f55', '9658b9c3c', '6ef6fc905', '6efc51980', '4cf91995d', '1bb0c013a', '4e6891a56', '5b3ca5452', 'a099be857', 'cea58b404', '718b1c7b8', '0adcb2a1a', 'c2d23766d', '4a654c127', 'c7ba07095', '7939edac8', '56fd2d9c7', '2c2c2a3d6', 'a4c89beca', '1b99d2fd6', '6f01adf08', '3cbee7440', '27f0a8188', 'afd82a1d7', 'd5b96f991', 'fae71c97b', '602e4b162', 'abcb46fc1', 'd8f09768d', '9182c159f', '4d77c47b6', '1da6549

100%|██████████| 3373/3373 [00:08<00:00, 413.16it/s]


In [ ]:
Configuration Files

For a yolo model, we need two config files with .yaml as extension;
 one contains the location of training & validation folders, 
 the number of classes,
  and class names, 
  second YAML file contains yolo model architecture.
First YAML File

%%writefile ./convertor/wheat0.yaml

# COCO 2017 dataset http://cocodataset.org - first 128 training images
# Download command:  python -c "from yolov5.utils.google_utils import gdrive_download; gdrive_download('1n_oKgR81BJtqk75b00eAjdv03qVCQn2f','coco128.zip')"
# Train command: python train.py --data ./data/coco128.yaml
# Dataset should be placed next to yolov5 folder:
#   /parent_folder
#     /coco128
#     /yolov5


# train and val datasets (image directory or *.txt file with image paths)
train: ./convertor/fold0/images/train2017/
val: ./convertor/fold0/images/val2017/

# number of classes
nc: 1

# class names
names: ['wheat']

Second YAML File

%%writefile ./convertor/yolov5x.yaml



# parameters
nc: 1  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# parameters
nc: 1  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [116,90, 156,198, 373,326]  # P5/32
  - [30,61, 62,45, 59,119]  # P4/16
  - [10,13, 16,30, 33,23]  # P3/8

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
  ]

# YOLOv5 head
head:
  [[-1, 3, BottleneckCSP, [1024, False]],  # 9

   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],  # 18 (P3/8-small)

   [-2, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],  # 22 (P4/16-medium)

   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],  # 26 (P5/32-large)

   [[], 1, Detect, [nc, anchors]],  # Detect(P5, P4, P3)
  ]

#Model Training

!python /content/train.py --img 1024 --batch 16 --epochs 10 --data /content/convertor/wheat0.yaml --name yolov5x_fold0 

